<a href="https://colab.research.google.com/github/Shreyasi3521/MNIST-classifier-with-pytorch/blob/master/MNISTtorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import numpy as np
from torchvision import datasets,transforms
from torch.nn import Conv2d, MaxPool2d, Linear, Module, Dropout, CrossEntropyLoss
from torch.nn import functional as F

In [0]:
trainset = torchvision.datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),]))
testset = torchvision.datasets.MNIST(root='./data/MNIST', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),]))
train = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw


Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw


Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
class Net(Module):
    def __init__(self):
        super(Net,self).__init__()
        self.L1=Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
        self.L2=Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
        self.L3=MaxPool2d(kernel_size=(2,2))
        self.L4=Dropout(p=0.25)
        self.L5=Linear(in_features=9216, out_features=128)
        self.L6=Dropout()
        self.L7=Linear(in_features=128, out_features=10)
    def forward(self,x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        x = F.relu(x)
        x = self.L3(x)
        x = self.L4(x)
        x = x.view(-1,9216)
        x = self.L5(x)
        x = F.relu(x)
        x = self.L6(x)
        x = self.L7(x)
        x = F.softmax(x, dim=1)
        return x


In [0]:
net=Net()

In [0]:
import torch.optim as optim
criterion = CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
import time
start_time=time.time()

In [0]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(epoch + 1, 'loss =', running_loss/len(train))
print('Finished Training')

1 loss = 2.2572363581992922
2 loss = 1.8103078934175374
3 loss = 1.643151820976851
4 loss = 1.6042674488858628
5 loss = 1.5855313491211263
6 loss = 1.5718453323154815
7 loss = 1.5609111423685607
8 loss = 1.5520747016742031
9 loss = 1.5453452606445182
10 loss = 1.538762519481594
11 loss = 1.5332516398765386
12 loss = 1.529696827631261
13 loss = 1.5263972034586517
14 loss = 1.5234006117147678
15 loss = 1.5191234362913346
16 loss = 1.5164269527900955
17 loss = 1.514181911309899
18 loss = 1.5122588617461068
19 loss = 1.5108834754175215
20 loss = 1.5080575034308281
21 loss = 1.5070180992073596
22 loss = 1.5058837446576752
23 loss = 1.504578437632335
24 loss = 1.5027491170714404
25 loss = 1.5016142827615555
Finished Training


In [0]:
gpu_time=time.time()-start_time
gpu_time

3263.4106233119965

In [0]:
cpu_time=time.time()-start_time
cpu_time

3446.9949724674225

In [0]:
correct_count, all_count = 0, 0
for images,labels in test:
    with torch.no_grad():
        pred = net(images)
    pred = pred.numpy()
    for i in range(len(labels)):
        pred_label = np.argmax(pred[i])
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
            correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count*100/all_count),'%')

Number Of Images Tested = 10000

Model Accuracy = 96.25 %
